In [1]:
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT

In [24]:
import wandb

In [4]:
out_dir = r'C:\Users\alireza\Documents\git\nanoGPT\out-moSeq-syll_e1'
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
device = 'cuda'
checkpoint = torch.load(ckpt_path, map_location=device)

C:\Users\alireza\AppData\Local\Temp\ipykernel_10812\4173216363.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


In [6]:
checkpoint.keys()

dict_keys(['model', 'optimizer', 'model_args', 'iter_num', 'best_val_loss', 'config'])

In [ ]:
"""
Sample from a trained model
"""
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
import numpy as np
from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = r'C:\Users\alireza\Documents\git\nanoGPT\out-moSeq-syll_e1' # ignored if init_from is not 'resume'
data_type = 'numpy_array' # or 'numpy_array' for the moSeq dataset
start = "\n"#r'FILE:C:\Users\alireza\Documents\git\nanoGPT\data\moSeq_syllabus_small\val_data_sample.npy' #"\n" or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt" or a numpy array "FILE:prompt.npy"
num_samples = 1 # number of samples to draw
max_new_tokens = 1000 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
sample_file_name = 'samples' # where to save the samples
print_samples = True # print the samples to the console
save_samples = True # save the samples to a file
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
#exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    if data_type == 'text':
        encode = lambda s: [stoi[c] for c in s]
        decode = lambda l: ''.join([itos[i] for i in l])
    elif data_type == 'numpy_array':
        encode = lambda s: [stoi[c] for c in s]
        decode = lambda l: np.array([itos[i] for i in l])
    else:
        # TODO: you really need abstract this and make it more general - i.e., make it independent of the 
        # data_type and to the encoder/decoder then transter it the the data_type
        raise ValueError(f"Unknown data_type: {data_type}")
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

# if the data is a natrual language text, so the data is a string 
if data_type == 'text':
    # encode the beginning of the prompt
    if start.startswith('FILE:'):
        with open(start[5:], 'r', encoding='utf-8') as f:
            start = f.read()
    start_ids = encode(start)
    x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

elif data_type == 'numpy_array':
    if start.startswith('FILE:'):
        start = np.load(start[5:], allow_pickle=True)
    else:
        start = decode([1])
    start_ids = encode(start)
    x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
else:
    raise ValueError(f"Unknown data_type: {data_type}")
    # run generation
"""
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            if print_samples:
                print(decode(y[0].tolist()))
                print('---------------')
            if save_samples:
                if data_type == 'text':
                    # save the samples to a text file
                    file_name = f"{sample_file_name}_{k}.txt"
                    file_path = os.path.join(out_dir, file_name)
                    # rewrite the file if it already exists
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(decode(y[0].tolist()))
                elif data_type == 'numpy_array':
                    # save the samples to a numpy file
                    file_name = f"{sample_file_name}_{k}.npy"
                    file_path = os.path.join(out_dir, file_name)
                    # rewrite the file if it already exists
                    np.save(file_path, y[0].cpu().numpy())
                else:
                    raise ValueError(f"Unknown data_type: {data_type}")
"""

C:\Users\alireza\AppData\Local\Temp\ipykernel_9300\3443039349.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=device)


using fast attention with Flash Attention CUDA kernels
using fast attention with Flash Attention CUDA kernels
using fast attention with Flash Attention CUDA kernels
using fast attention with Flash Attention CUDA kernels
using fast attention with Flash Attention CUDA kernels
using fast attention with Flash Attention CUDA kernels
number of parameters: 10.64M
Loading meta from data\moSeq_syllabus_small\meta.pkl...


'\nwith torch.no_grad():\n    with ctx:\n        for k in range(num_samples):\n            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)\n            if print_samples:\n                print(decode(y[0].tolist()))\n                print(\'---------------\')\n            if save_samples:\n                if data_type == \'text\':\n                    # save the samples to a text file\n                    file_name = f"{sample_file_name}_{k}.txt"\n                    file_path = os.path.join(out_dir, file_name)\n                    # rewrite the file if it already exists\n                    with open(file_path, \'w\', encoding=\'utf-8\') as f:\n                        f.write(decode(y[0].tolist()))\n                elif data_type == \'numpy_array\':\n                    # save the samples to a numpy file\n                    file_name = f"{sample_file_name}_{k}.npy"\n                    file_path = os.path.join(out_dir, file_name)\n                    # rew

In [28]:
x

tensor([[1]], device='cuda:0')